# 5. Zadania domowe
Patryk Żywica (2025)


W ramach tego modułu do zdobycia jest 4 punktów. 

> **Uwaga**: Łącznie można wykonać zadania za znacznie więcej niż dostępnych 4 punktów. Uzyskany wynik zostanie jednak obcięty do wartości maksymalnej.


## Zadanie 5.1 (2 punkty)
Zaproponuj architekturę dla wymyślonego przez siebie systemu informatycznego korzystającego z zarządzanej usługi bazy danych (DBaaS, np. Amazon DynamoDB, Azure Cosmos DB). System powinien zawierać przynajmniej dwie odseparowane usługi jednocześnie korzystające z bazy (np.: jeden proces zapisujący do bazy duże ilości danych, drugi, który je odczytuje i generuje API JSON). Jakie korzyści niesie wykorzystanie zarządzanej bazy danych dla tak zaprojektowanego systemu? Jakie są ryzyka? Opracuj dokument przedstawiający architekturę takiego systemu, w którym przedstawisz obszerną analizę problemu, wzbogaconą o stosowne diagramy, porównanie z klasyczną architekturą w modelu BYODB (buy your own DB) oraz wstępny kosztorys zakładający różne obciążenia systemu.

> **Wskazówka**: Zadanie można realizować we dwie osoby, obie otrzymują wtedy taką samą ilość punktów.

Raport powinien:
* mieć przynajmniej 5 stron, 
* zawierać przynajmniej 1 diagram architektury,
* przedstawiać przynajmniej po jednej wadzie i zalecie wynikającej z użycia zarządzanej bazy danych,
* zawierać porównanie z modelem BYODB,
* zawierać fragment kodu źródłowego pokazującego jak system ma wykonywać zapytania do bazy danych w wybranym języku programowania,
* zawierać analizę kosztów wybranego rozwiązania.

## Sposób sprawdzania zadania
Zadanie sprawdzane jest ręcznie. Przygotuj raport w formacie PDF i wyślij go prowadzącemu zajęcia. 

## Zadanie 5.2 (4 punkty)
Zadanie ma na celu przedstawienie usług SaaS z zakresu sztucznej inteligencji i przetwarzania obrazów.

Zadanie polega na przygotowaniu prostego systemu informatycznego pozwalającego na odczytywanie treści z zeskanowanych dokumentów. Na potrzeby zadania przyjmiemy, że dokumenty to polskie faktury VAT. Celem jest wyekstrahowanie nazwy sprzedawcy, jego numeru NIP oraz kwoty łącznej brutto faktury.

Webserwis ma obsługiwać tylko jeden endpoint HTTP przyjmujący zapytania POST. Komunikacja z webserwisem powinna wyglądać następująco:
```bash
curl -X POST <nazwa_serwera_z_zadaniem>:<port>/invoice -F "file=@dane/train/AFV 1_05_2021.pdf"
```
Odpowiedź z serwera:
```json
{
    "vat_id": "123456789",
    "address": "ul. Uniwersytetu Poznańskiego 4, 61-614 Poznań",
    "total": 123.45
}
```

### Sposób sprawdzania zadania
Zadanie będzie sprawdzane półautomatycznie. Do skryptu sprawdzającego należy podać adres URL webserwisu. Skrypt sprawdzający odpyta webserwis o 10 losowo wybranych dokumentów z bazy testowej i przygotuje zestawienie wyników. Ostateczna decyzja o zaliczeniu zadania należy do prowadzącego zajęcia.

Poniższy skrypt pozwala na odpytanie webserwisu i zebranie odpowiedzi w ustrukturyzowanej formie na potrzeby ręcznej oceny.

In [6]:
WEBSERVICE_URL = "https://s504297-invoice-ocr-d2df690de0f8.herokuapp.com"

import requests
import glob
import sys

txtfiles = []
for file in glob.glob("./dane/Z02/train/*.pdf"):
    print(f"Testowanie pliku {file}")
    files = {'file': open(file, 'rb')}
    response = requests.post(WEBSERVICE_URL + "/invoice", files=files)
    try:
        data = response.json()
        #print(data)
        if {"vat_id", "address", "total"}.issubset(data) and data["vat_id"] and data["address"] and isinstance(data["total"], (int, float)):
            print(f'OK: {data["vat_id"]}	{data["total"]}	{data["address"]}')
        else:
            print("NIEPOPRAWNA ODPOWIEDŹ")
    except Exception as exc:
        print(f"Błędna odpowiedź z serwera ({exc})", file=sys.stderr)


Testowanie pliku ./dane/Z02/train/BFV 2_05_2021.pdf
OK: 7831702323	158.66	FUNDACJA MUZEUM MOTORYZACJI
AUTOMOBILKLUBU WIELKOPOLSKI KUZNIA
-
DZIEDZICTWA
ul. Towarowa 35/37
61-896 Poznan
Testowanie pliku ./dane/Z02/train/BFV 4_05_2021.pdf
OK: 5741997874	246.0	FUNDACJA MUZEUM MOTORYZACJI
AUTOMOBILKLUBU WIELKOPOLSKI KUZNIA
-
DZIEDZICTWA
ul. Towarowa 35/37
61-896 Poznan
Testowanie pliku ./dane/Z02/train/CFV 1_05_2021.pdf
OK: 7831702323	109.46	FUNDACJA MUZEUM MOTORYZACJI
AUTOMOBILKLUBU WIELKOPOLSKI KUZNIA
-
DZIEDZICTWA
ul. Towarowa 35/37
61-896 Poznan
Testowanie pliku ./dane/Z02/train/AFV 4_05_2021.pdf
OK: 7831702323	246.0	FUNDACJA MUZEUM MOTORYZACJI AUTOMOBILKLUBU
WIELKOPOLSKI KUZNIA DZIEDZICTWA
-
ul. Towarowa 35/37
61-896 Poznan
NIP: 7831702323
Testowanie pliku ./dane/Z02/train/CFV 2_05_2021.pdf
OK: 7831702323	158.66	FUNDACJA MUZEUM MOTORYZACJI
AUTOMOBILKLUBU WIELKOPOLSKI KUZNIA
-
DZIEDZICTWA
ul. Towarowa 35/37
61-896 Poznan
Testowanie pliku ./dane/Z02/train/AFV 3_05_2021.pdf
OK: 7831702323

## Zadanie 5.3 (2 punkty, wymaga ukończenia zadania 5.2)

Opracuj prosty interfejs webowy dla systemu z zadania 5.2. Powinien być opracowany jako statyczna strona www (html + javascript + css), a następnie wdrożony z wykorzystaniem AWS S3 + Cloudfront lub Azure Storage + CDN.

> **Wskazówka**: Konfiguracja HTTPS wykracza poza zakres tego zadania. Wystarczy jeśli strona będzie działała przez HTTP.

> **Wskazówka**: Następująca dokumentacja może posłużyć jako dobry punkt startowy dla [Azure](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-blob-static-website) oraz [AWS](https://aws.amazon.com/premiumsupport/knowledge-center/cloudfront-serve-static-website/).


### Sposób sprawdzania zadania
Zadanie będzie sprawdzane ręcznie. Przygotuj się do prezentacji aplikacji prowadzącemu zajęcia.